# Travel Time 

### Goal of the notebook

The purpose of this Jupyter notebook is to request the travel time in second from the origin to destination using google API.
***
**Outputs:** 

- travel_time


**Inputs:** 

- ori_centroid_id
- des_centroid_id
- select_time

External centroid data:

- External_centroid_googlemap.xlsx




**Dependent scripts:** 
None

 **IMPORTANT:** Run the iPython cell below in order to import the necessary packages.

In [221]:
import numpy as np
import scipy
import matplotlib
import pandas as pd
import numpy as np
import json as simplejson
import urllib
from urllib import request
import requests, json 
from bs4 import BeautifulSoup
from datetime import datetime
import time

### Load external centroid data

This function loads the external centroid data with longtitue and latitute from dropbox into python notebook

In [222]:
# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from fremontdropbox import get_dropbox_location

path_dropbox = get_dropbox_location()

path_data=os.path.join(path_dropbox, "Data Collection/Data processing/Data in process/Demand/External_Centroids_GoogleMap.csv")
external_data=pd.read_csv(path_data)
external_data=external_data.set_index('Centroid_I')
external_data

,lon/lau,NB/EB,SB/WB
Centroid_I,,,
4,"37.5111060469967,-121.954604452945","37.510970,-121.954368","37.511344,-121.954390"
5,"37.5330845192806,-121.9545231","37.533058,-121.954249","37.533219,-121.954270"
6,"37.5413236280827,-121.9559944","37.541187,-121.956359","37.541341,-121.956370"
13,"37.5495438286178,-121.91921","37.549688,-121.919436","37.549534,-121.919107"
15,"37.5275111832535,-121.8961378","37.527723,-121.896026","37.527530,-121.895941"
16,"37.5208527574847,-121.9111971","37.520865,-121.911228","37.520855,-121.911188"
19,"37.4908772090958,-121.9088049","37.490923,-121.908900","37.490856,-121.908919"
20,"37.4898018766374,-121.9179774","37.489853,-121.917752","37.489776,-121.918149"
12,"37.5534590835457,-121.9455607","37.553485,-121.945378","37.553442,-121.945625"


### Google API key

This cell is to define the Google API key that we will be using

In [223]:
#This is my own google API key which does not work for now
Google_API_KEY=

### Function to request Travel Time

This function is defined to request the travel time using Google API

In [236]:
def GetTravelTime(ori_external_id, des_external_id, time):
    orig_coord_NB_EB = external_data.loc[ori_external_id,'NB/EB']
    orig_coord_SB_WB = external_data.loc[ori_external_id,'SB/WB']
    dest_coord_NB_EB = external_data.loc[des_external_id,'NB/EB']
    dest_coord_SB_WB = external_data.loc[des_external_id,'SB/WB']
    l=[]
    url ='https://maps.googleapis.com/maps/api/distancematrix/json?'
    
    r = url + "origins=" + orig_coord_NB_EB + "&destinations=" + dest_coord_NB_EB + \
         "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY

    result= simplejson.load(urllib.request.urlopen(r))
    driving_time1 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time1)
    
    r = url + "origins=" + orig_coord_NB_EB + "&destinations=" + dest_coord_SB_WB + \
         "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY
    result= simplejson.load(urllib.request.urlopen(r))
    driving_time2 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time2)
    
    r = url + "origins=" + orig_coord_SB_WB + "&destinations=" + dest_coord_NB_EB + \
         "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY
    result= simplejson.load(urllib.request.urlopen(r))
    driving_time3 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time3)
    
    r = url + "origins=" + orig_coord_SB_WB + "&destinations=" + dest_coord_SB_WB + \
        "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY 
    result= simplejson.load(urllib.request.urlopen(r))
    driving_time4 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time4)
    return min(l)


### Function to download the travel time

This function is download the travel time between different external

In [225]:
l=list(external_data.index)

In [227]:
#The timeStamp is used on March 15th from 00：00 to 22:00 UTC time
ts={}
for i in range(12):
    ts[str(i*2)+":00"]=1584255600+7200*i
ts

{'0:00': 1584255600,
 '2:00': 1584262800,
 '4:00': 1584270000,
 '6:00': 1584277200,
 '8:00': 1584284400,
 '10:00': 1584291600,
 '12:00': 1584298800,
 '14:00': 1584306000,
 '16:00': 1584313200,
 '18:00': 1584320400,
 '20:00': 1584327600,
 '22:00': 1584334800}

In [228]:
df=pd.DataFrame([], columns = ['ori_external_id', 'des_external_id',"time","travel_time"])

In [229]:
def download_travel(df):
    ids=list(external_data.index)
    for i in ids:
        for j in ids:
            if i != j:
                for k in ts:
                    data = [[i,j,datetime.fromtimestamp(ts[k]).hour,GetTravelTime(i,j,str(ts[k]))]]
                    df1=pd.DataFrame(data, columns=['ori_external_id', 'des_external_id',"time","travel_time"])
                    df=df.append(df1)
    return df

In [233]:
df = download_travel(df)
df

[4, 5, 6, 13, 15, 16, 19, 20, 12, 22, 21]
[262]
[272]
[266]
[264]
[278]
[329]
[351]
[366]
[322]
[345]
[338]
[298]
[413]
[390]
[382]
[386]
[412]
[472]
[512]
[526]
[495]
[496]
[466]
[440]
[321]
[315]
[328]
[330]
[327]
[352]
[371]
[376]
[365]
[373]
[359]
[336]
[829]
[835]
[840]
[854]
[855]
[892]
[890]
[924]
[905]
[912]
[891]
[870]
[405]
[409]
[419]
[420]
[422]
[468]
[486]
[487]
[471]
[471]
[456]
[436]
[421]
[409]
[418]
[433]
[432]
[461]
[499]
[513]
[500]
[510]
[469]
[446]
[236]
[228]
[236]
[246]
[242]
[263]
[287]
[297]
[283]
[297]
[272]
[258]
[487]
[487]
[480]
[503]
[507]
[552]
[571]
[585]
[561]
[571]
[545]
[518]
[359]
[394]
[345]
[379]
[371]
[399]
[421]
[441]
[418]
[424]
[404]
[382]
[279]
[272]
[274]
[284]
[278]
[299]
[326]
[348]
[328]
[329]
[308]
[291]
[257]
[251]
[249]
[257]
[251]
[277]
[283]
[291]
[275]
[292]
[286]
[265]
[162]
[180]
[175]
[164]
[167]
[185]
[194]
[195]
[183]
[190]
[182]
[177]
[251]
[245]
[250]
[247]
[243]
[259]
[269]
[274]
[262]
[276]
[270]
[260]
[769]
[766]
[767]
[772

,ori_external_id,des_external_id,time,travel_time
0,4,5,0,262
0,4,5,2,272
0,4,5,4,266
0,4,5,6,264
0,4,5,8,278
...,...,...,...,...
0,21,22,14,136
0,21,22,16,119
0,21,22,18,116
0,21,22,20,118
